<a href="https://colab.research.google.com/github/sandy273040/Text_Mining/blob/main/build_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tqdm requests multitasking retry bs4 pandas numpyxlrd openpyxl
!pip install zhon
!pip install transformers
!pip install ckip-transformers
!pip install tensorflow
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached retry-0.9.2-py2.py3-none-any.whl (8.0 kB)
ERROR: Could not find a version that satisfies the requirement numpyxlrd (from versions: none)
ERROR: No matching distribution found for numpyxlrd
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import re
from tqdm.notebook import tqdm
import string
from zhon.hanzi import punctuation
from zhon.zhuyin import characters
from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger, CkipNerChunker
import torch
import pickle
import glob


In [ ]:
! /opt/bin/nvidia-smi

Tue Jan 10 14:25:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
print(torch.cuda.is_available())

True


###Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [ ]:
def remove_punctuation(x):
  for i in string.punctuation:
    x = x.replace(i, '')
  for i in punctuation:
    x = x.replace(i, '')
  for i in characters:
    x = x.replace(i, '')
  return x

###Read File

In [ ]:
def read_files():
  files = glob.glob("/content/drive/Shareddrives/文字探勘/final_project/restaurant_comments/*.csv")
  restaurant_info = pd.read_csv("/content/drive/Shareddrives/文字探勘/final_project/restaurant_df.csv", index_col=[0])

  # restaurant_df = pd.DataFrame(columns=['restaurant_id', 'restaurant_name', 'length', 'ave_rating'])



  return files, restaurant_info

In [ ]:
files, restaurant_info = read_files()

In [ ]:
def build_df(files, restaurant_info):
  '''build dataframes to store restaurant info and comments of each users respectively'''
  restaurant_df = pd.DataFrame(columns=['restaurant_id', 'restaurant_name', 'length', 'ave_rating', 'popularity'])
  comment_df = pd.DataFrame()#comment

  for id, f in enumerate(files):
    #append new comment to df
    read = pd.read_csv(f, index_col = 0).dropna()
    comment_df = comment_df.append(read, ignore_index = True)

    #update restaurant df
    restaurant_name = f.split('/')[-1].split('.')[-2]
    ave_rating = restaurant_info[restaurant_info['restaurant_name'] == restaurant_name]['ave_rating'].tolist()[0]
    popularity = restaurant_info[restaurant_info['restaurant_name'] == restaurant_name]['comment_amount'].tolist()[0]
    
    restaurant_df.loc[len(restaurant_df)] = [id, restaurant_name, len(read), ave_rating, popularity]
    
  return restaurant_df, comment_df

In [ ]:
restaurant_df, df = build_df(files, restaurant_info)

In [ ]:
restaurant_df

In [ ]:
df

In [ ]:
# '''the complete dataframe that holds the comment info'''
# df = pd.DataFrame()

In [ ]:
# from math import nan
# restaurant_df = pd.DataFrame(columns=['restaurant_id', 'restaurant_name', 'length', 'ave_rating'])

# for id, f in enumerate(files):
#   #append new comment to df
#   read = pd.read_csv(f, index_col = 0).dropna()
#   df = df.append(read, ignore_index = True)
#   #update restaurant df
#   avg_rating = read['rating'].mean()
#   # restaurant_df.loc[len(restaurant_df)] = [id, f.split('/')[-1].split('.')[-2], len(read), avg_rating]
#   restaurant_name = f.split('/')[-1].split('.')[-2]
#   # restaurant_rating = restaurant_info[restaurant_info['name'] == restaurant_name]['rating'].values
#   # if restaurant_rating.shape == (1,): print(restaurant_name)
#   # print(restaurant_rating.shape == (0,))
#   restaurant_df.loc[len(restaurant_df)] = [id, restaurant_name, len(read), avg_rating]

# df

In [ ]:
# restaurant_df
# restaurant_df.to_csv('/content/drive/Shareddrives/文字探勘/final_project/restaurant_df.csv')

In [ ]:
# restaurant_df = pd.read_csv('/content/drive/Shareddrives/文字探勘/final_project/new_restaurant_df.csv', index_col=[0]).drop(['Unnamed: 0.1', 'ave_rating'], axis=1)
# restaurant_df = restaurant_df.rename({'total_rating': 'ave_rating'}, axis=1)
# restaurant_df

In [ ]:
def preprocessing_comment(df):
  comment = df['comment']
  comment = comment.apply(lambda x: str(x))
  # remove emoji
  comment = comment.apply(lambda x: remove_emojis(x))
  #remove punctaution mark
  comment = comment.apply(lambda x: remove_punctuation(x))
  comment = comment.str.replace('[a-zA-Z0-9]', '')
  comment = comment.str.replace('\n', '')

  return list(comment)

In [ ]:
comment_lst = preprocessing_comment(df)
comment_lst

###word segmentation

In [ ]:
def word_segmentation(comment_lst):
  '''input a list of comment strings to segment word'''
  print("Running pipeline ... WS")
  ws_driver  = CkipWordSegmenter(model="bert-base", device = 0)
  ws = ws_driver(comment_lst)# a list of lists of segmented comments
  
  return [' '.join(sentence) for sentence in ws] # join each word segment using space to form a list of strings

In [ ]:
comment_lst = word_segmentation(comment_lst)

In [ ]:
with open('/content/drive/Shareddrives/文字探勘/final_project/comment.pkl', 'wb') as f:
  pickle.dump(comment_lst, f)

In [ ]:
import gc
gc.collect()
del ws_driver, ws

###Vectorization

In [ ]:
with open('/content/drive/Shareddrives/文字探勘/final_project/comment.pkl', 'rb') as f:
  comment_lst = pickle.load(f)
comment_lst

['同 桌 每 人 點 套餐 有 薯條 吃到飽 薯條 現 炸 十分 酥脆 是 有 厚度 的 薯條 很 好吃 不 是 麥當勞 那 種 細長 的 打卡 有 另外 送 雞米花 打 一 個 送 一 小 籃 炸 得 外 酥 內嫩 不會 因為 是 送 的 就 隨便 炸 給 你 很 值得 來 一 籃  ',
 '  位於 師大 附近 街道 轉角 上 沒想到 在 轉角處 就 可以 看到 如此 浪漫 的 餐廳 一 進 門 可以 感受到 優雅 時尚 浪漫 的 氛圍  ',
 '第一 次 嘗 鮮 點 了 羊排 跟 漢堡排 皆 點 套餐 所以 有 附 薯條 吃 到 飽 薯條 上桌 時 很 驚艷 居然 是 咖哩 口味 的 同 桌 友人 吃 得 不亦樂乎 最後 續 了 兩 次 第三 次 店員 直接 幫 我們 薯條 炸裂 小 籠子 塞爆 一 上桌 就 快 把 我 笑死 大概 看 這 桌 太 會 吃 薯條 麵包 熱度 及 酥軟度 剛好 沙拉 及 湯品 中規中矩 加 點 紅酒 也 有 在 要求 時間 內 上  ',
 '第一 次 吃到 沙拉 搭配 橘片乾 口味 很 和諧 也 很 爽口 濃湯 的 部分 口味 濃郁 且 香 再 搭配 一些 黑 胡椒 ⋯ 好吃 主食 的 鱸魚 外層 微脆 肉質 香嫩 應該 很 適合 長輩 享用 餐點 好吃 份量 非常 多 建議 可以 一 個 人 點 套餐 一 個 人 單 點 這樣 比較 不 容易 膩 且 份量 較 剛好',
 '很 適合 和 三五 好友 聊天 聚餐 的 地方 一 個 戴 眼鏡 的 女 服務生 人 很 親切 和善 空間 很 大 很 舒適  ',
 '點餐 內容 安格斯 牛排 ️ 分 熟肉 吃起來 很 嫩 本身 已 有 調味 好吃 戰斧 豬排 ️ 朋友 點 的 肉質 本身 無可 挑剔 價格 也 很 合宜  ',
 '環境 舒服 服務 周到 餐點 可以 幫忙 客製 牛排 厚實 肉味 足 雞米花 吃 得 出 特別 用 香料 醃漬 過 味道 不錯 就是 炸 得 稍 乾 燉飯 香 入味 套餐 送 薯條',
 '第一 次 來 此 餐廳 用餐 服務 人員 親切 快速 我 在 猶豫 時 也 很 有 耐心 在 旁 等候 對 下班 的 我 來 說 真的 是 感覺 很 棒 也 辛苦 店員 了 牛排 好吃 海鱸魚 也 很 棒 很 幸福 的 夜晚',
 '今天 點 的 是 聖誕節 套

In [ ]:
# datasets = [doc.split() for doc in comment_lst]

In [ ]:
def build_stopwords():
  stopwords = dict()
  with open('/content/drive/Shareddrives/文字探勘/final_project/stop_word.txt') as f:
      lst = f.read().split('\n')
      for s in lst:
        stopwords[s] = s
  return stopwords

In [ ]:
stopwords = build_stopwords()
stopwords

{'口味': '口味',
 '真的': '真的',
 '可以': '可以',
 '不錯': '不錯',
 '好吃': '好吃',
 '認識 ': '認識 ',
 '本地 ': '本地 ',
 '如同 ': '如同 ',
 '這一 ': '這一 ',
 '~~ ': '~~ ',
 '盡心 ': '盡心 ',
 '轉貼 ': '轉貼 ',
 '正是 ': '正是 ',
 '強調': '強調',
 '!': '!',
 '"': '"',
 '#': '#',
 '$': '$',
 '%': '%',
 '&': '&',
 "'": "'",
 '(': '(',
 ')': ')',
 '*': '*',
 '+': '+',
 ',': ',',
 '-': '-',
 '--': '--',
 '.': '.',
 '..': '..',
 '...': '...',
 '......': '......',
 '...................': '...................',
 './': './',
 '.一': '.一',
 '.數': '.數',
 '.日': '.日',
 '/': '/',
 '//': '//',
 '0': '0',
 '1': '1',
 '2': '2',
 '3': '3',
 '4': '4',
 '5': '5',
 '6': '6',
 '7': '7',
 '8': '8',
 '9': '9',
 ':': ':',
 '://': '://',
 '::': '::',
 ';': ';',
 '<': '<',
 '=': '=',
 '>': '>',
 '>>': '>>',
 '?': '?',
 '@': '@',
 'A': 'A',
 'Lex': 'Lex',
 '[': '[',
 ']': ']',
 '^': '^',
 '_': '_',
 '`': '`',
 'exp': 'exp',
 'sub': 'sub',
 'sup': 'sup',
 '|': '|',
 '}': '}',
 '~': '~',
 '~~~~': '~~~~',
 '·': '·',
 '×': '×',
 '×××': '×××',
 'Δ': 'Δ',
 'Ψ': 'Ψ',


In [ ]:
def stopowords_filtering(stopwords, datasets):
  data = list()
  for doc in datasets:
    temp = list()
    for w in doc:
      if stopwords.get(w) is None:
        temp.append(w)
    data.append(temp)
  # datasets = data

  return data

In [ ]:
import itertools
from collections import Counter
from nltk.util import ngrams

def bigram(restaurant_df, datasets):
  comments = list()
  start = 0
  print(restaurant_df['length'])
  for length in restaurant_df['length']:
    lst = datasets[start:start + length]
    lst = list(itertools.chain.from_iterable(lst))

    # obtain all bigrams
    news_bigrams = ngrams(lst, 2)
    # count occurrences of each bigram
    news_bigrams_freq = Counter(news_bigrams)
    # review top 5 frequent bigrams in the news
    print("Top 5 bigrams:\n{}".format(news_bigrams_freq.most_common(5)))#ranked_wordtuple_first word
    top_bigram = news_bigrams_freq.most_common(5)
    for rank in top_bigram:
      # for tup in rank:
      #   print(tup[0])
      lst.remove(lst[lst.index(rank[0][0])])
      lst.remove(lst[lst.index(rank[0][1])])
      lst.append(rank[0][0] + rank[0][1])

    lst = [word for word in lst if len(word) > 1]
    print(lst)
    comments.append(lst)

    start += length
    
  return comments

In [ ]:
def word_processing(stopwords, comment_lst, restaurant_df):
  '''since LDA is quite sensitive to the cleaness of data, we further clean our datasets using stopwords and bigram'''
  datasets = [doc.split() for doc in comment_lst]
  
  datasets = stopowords_filtering(stopwords, datasets)
  datasets = bigram(restaurant_df, datasets)
  return datasets

In [ ]:
comments = word_processing(stopwords, comment_lst, restaurant_df)
comments

0      634
1      840
2      840
3      840
4      840
      ... 
101    840
102    840
103    840
104    840
105    445
Name: length, Length: 106, dtype: object
Top 5 bigrams:
[(('氣氛', '佳'), 45), (('服務', '人員'), 36), (('用餐', '環境'), 32), (('椒麻', '牛排'), 30), (('環境', '舒適'), 29)]
['套餐', '薯條', '吃到飽', '薯條', '酥脆', '厚度', '薯條', '麥當勞', '細長', '打卡', '雞米花', '內嫩', '隨便', '值得', '位於', '師大', '街道', '轉角', '沒想到', '轉角處', '浪漫', '餐廳', '感受到', '優雅', '時尚', '浪漫', '氛圍', '第一', '羊排', '漢堡排', '套餐', '薯條', '薯條', '上桌', '驚艷', '咖哩', '友人', '第三', '店員', '薯條', '炸裂', '籠子', '塞爆', '上桌', '笑死', '薯條', '麵包', '熱度', '酥軟度', '沙拉', '湯品', '中規中矩', '紅酒', '時間', '第一', '吃到', '沙拉', '搭配', '橘片乾', '和諧', '爽口', '濃湯', '濃郁', '搭配', '胡椒', '主食', '鱸魚', '外層', '微脆', '肉質', '香嫩', '適合', '長輩', '享用', '餐點', '份量', '建議', '套餐', '容易', '份量', '適合', '三五', '好友', '聊天', '聚餐', '地方', '眼鏡', '服務生', '親切', '和善', '空間', '點餐', '內容', '安格斯', '熟肉', '吃起來', '調味', '戰斧', '豬排', '朋友', '肉質', '無可', '挑剔', '價格', '合宜', '舒服', '周到', '餐點', '幫忙', '客製', '牛排', '厚實', '肉味', '雞米花', '特別', '香料', '醃漬', '味道',

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from gensim import corpora
from gensim import models

dictionary = corpora.Dictionary(comments)#mapping word to its index
corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in comments]# convert docs into vectors' index (not yet build vector)
# for doc in corpus:
#    print([[dictionary[id], freq] for id, freq in doc])
model = models.TfidfModel(corpus, id2word = dictionary)

In [ ]:
def filter_tfidf(corpus, model):
  '''filter low tfidf word'''
  low_value = 0.025

  for i in range(0, len(corpus)):
      bow = corpus[i]
      low_value_words = [] #reinitialize to be safe. You can skip this.
      low_value_words = [id for id, value in model[bow] if value < low_value]
      new_bow = [b for b in bow if b[0] not in low_value_words]

      #reassign        
      corpus[i] = new_bow
  return corpus

In [ ]:
corpus = filter_tfidf(corpus, model)# a list of lists of (word index, tfidf value of each doc)
corpus

[[(10, 5),
  (24, 8),
  (39, 24),
  (45, 3),
  (47, 21),
  (66, 21),
  (73, 5),
  (95, 22),
  (96, 42),
  (98, 6),
  (100, 17),
  (102, 11),
  (183, 7),
  (184, 2),
  (190, 21),
  (209, 11),
  (217, 4),
  (218, 4),
  (242, 21),
  (254, 7),
  (255, 1),
  (259, 7),
  (263, 32),
  (270, 2),
  (284, 2),
  (289, 13),
  (322, 3),
  (326, 63),
  (334, 2),
  (336, 1),
  (347, 11),
  (348, 2),
  (357, 4),
  (365, 3),
  (387, 2),
  (388, 1),
  (407, 5),
  (409, 14),
  (426, 4),
  (428, 10),
  (431, 4),
  (471, 17),
  (473, 30),
  (479, 12),
  (504, 7),
  (511, 44),
  (514, 28),
  (542, 20),
  (547, 11),
  (561, 5),
  (575, 19),
  (576, 139),
  (586, 21),
  (588, 8),
  (589, 3),
  (598, 40),
  (599, 3),
  (602, 2),
  (607, 8),
  (620, 13),
  (623, 36),
  (632, 8),
  (633, 6),
  (638, 4),
  (669, 23),
  (689, 14),
  (690, 4),
  (698, 12),
  (707, 171),
  (708, 2),
  (709, 4),
  (721, 3),
  (724, 158),
  (739, 63),
  (776, 9),
  (780, 14),
  (795, 2),
  (798, 18),
  (802, 8),
  (804, 7),
  (832, 6)

In [ ]:
def index_word_mapping(dictionary, model):
    '''input a model contains (word index, tfidf) pair to find the words of each doc'''
    word = list()
    rest_word = list()
    for doc in model[corpus]:#document tfidf vector
      # print([(dictionary[id], freq) for id, freq in doc])
      word.append([dictionary[id] for id, freq in doc])
      rest_word.append([freq for id, freq in doc])
    return word, rest_word

    # for i in range(len(rest_word)):
    #   for j in np.argsort(rest_word[i])[0:5]:
    #     print(word[i][j])

    # print(np.argsort(rest_word[1])[-1:-5:-1])
    # print(word[0][24])

In [ ]:
word, rest_word = index_word_mapping(dictionary, model)

In [ ]:
tfidf = model[corpus]
tfidf

In [ ]:
feature_num = len(dictionary.token2id.keys())
feature_num # total number of feature names

52684

###LDA

####Compute coherence score to determine number of topics

In [ ]:
import gensim
from gensim.models import CoherenceModel

def get_coherence_values(corpus, dictionary, k):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=k, passes=20)
    
    coherence_model_lda = CoherenceModel(model=lda_model, corpus=corpus, dictionary=dictionary, texts=comments, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
coherence_lst = list()
for round in range(2, 40):
  coherence_lst.append(get_coherence_values(corpus, dictionary, round))
coherence_lst

KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt
def plot_coherence(coherence_lst, start, end):
  plt.plot([i for i in range(start, end)], coherence_lst, marker = 'o')
  plt.xlabel('number of topic')
  plt.ylabel('cv')
  plt.grid()
  plt.show()

In [ ]:
plot_coherence(coherence_lst, 2, 40)

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=dictionary, num_topics=25, passes=20)#好吃、可以、非常、不錯、真的
topic_list = lda.print_topics(25)
print("25個主題的單詞分佈為：\n")
for topic in topic_list:
    print(topic)

25個主題的單詞分佈為：

(0, '0.052*"服務" + 0.037*"炸雞" + 0.032*"餐點" + 0.028*"店員" + 0.025*"味道" + 0.024*"態度" + 0.021*"用餐" + 0.017*"韓式" + 0.017*"東西" + 0.017*"適合"')
(1, '0.046*"餐點" + 0.033*"服務" + 0.025*"適合" + 0.023*"環境" + 0.022*"用餐" + 0.020*"食物" + 0.020*"餐廳" + 0.018*"味道" + 0.018*"喜歡" + 0.016*"咖啡"')
(2, '0.081*"服務" + 0.053*"小菜" + 0.032*"店員" + 0.027*"態度" + 0.023*"用餐" + 0.023*"套餐" + 0.021*"味道" + 0.017*"八色" + 0.016*"海鮮" + 0.016*"生菜"')
(3, '0.103*"服務" + 0.036*"新鮮" + 0.032*"用餐" + 0.026*"店員" + 0.024*"態度" + 0.020*"親切" + 0.020*"品質" + 0.016*"東西" + 0.015*"客人" + 0.015*"肉質"')
(4, '0.091*"服務" + 0.069*"牛排" + 0.046*"餐點" + 0.035*"用餐" + 0.027*"餐廳" + 0.025*"環境" + 0.024*"麵包" + 0.021*"主餐" + 0.019*"沙拉" + 0.017*"美味"')
(5, '0.060*"餐點" + 0.048*"服務" + 0.027*"用餐" + 0.024*"環境" + 0.023*"份量" + 0.022*"適合" + 0.022*"味道" + 0.019*"燉飯" + 0.019*"餐廳" + 0.018*"套餐"')
(6, '0.101*"拉麵" + 0.065*"湯頭" + 0.043*"柑橘" + 0.040*"叉燒" + 0.033*"味道" + 0.028*"豚骨" + 0.022*"喜歡" + 0.021*"麵條" + 0.020*"排隊" + 0.018*"蛤蜊"')
(7, '0.038*"服務" + 0.034*"味道" + 0.031*"餐點"

In [ ]:
def get_topic_word(topic_list):
  '''
  input: top 10 word distribution of each topic
  output: formatted topic-word distribution using dictionary
  '''
  topic_words = dict()
  for topic_id, topic_word in enumerate(topic_list):
    topic_words[topic_id] = [item.split('*')[1] for item in topic_word[1].split('+')]
    # print(' '.join([item.split('*')[1] for item in i[1].split('+')]))
  return topic_words

In [ ]:
topic_words = get_topic_word(topic_list)
# restaurant_df['topic_words'] = pd.Series(topic_words)
# restaurant_df
topic_words

{0: ['"服務" ',
  '"炸雞" ',
  '"餐點" ',
  '"店員" ',
  '"味道" ',
  '"態度" ',
  '"用餐" ',
  '"韓式" ',
  '"東西" ',
  '"適合"'],
 1: ['"餐點" ',
  '"服務" ',
  '"適合" ',
  '"環境" ',
  '"用餐" ',
  '"食物" ',
  '"餐廳" ',
  '"味道" ',
  '"喜歡" ',
  '"咖啡"'],
 2: ['"服務" ',
  '"小菜" ',
  '"店員" ',
  '"態度" ',
  '"用餐" ',
  '"套餐" ',
  '"味道" ',
  '"八色" ',
  '"海鮮" ',
  '"生菜"'],
 3: ['"服務" ',
  '"新鮮" ',
  '"用餐" ',
  '"店員" ',
  '"態度" ',
  '"親切" ',
  '"品質" ',
  '"東西" ',
  '"客人" ',
  '"肉質"'],
 4: ['"服務" ',
  '"牛排" ',
  '"餐點" ',
  '"用餐" ',
  '"餐廳" ',
  '"環境" ',
  '"麵包" ',
  '"主餐" ',
  '"沙拉" ',
  '"美味"'],
 5: ['"餐點" ',
  '"服務" ',
  '"用餐" ',
  '"環境" ',
  '"份量" ',
  '"適合" ',
  '"味道" ',
  '"燉飯" ',
  '"餐廳" ',
  '"套餐"'],
 6: ['"拉麵" ',
  '"湯頭" ',
  '"柑橘" ',
  '"叉燒" ',
  '"味道" ',
  '"豚骨" ',
  '"喜歡" ',
  '"麵條" ',
  '"排隊" ',
  '"蛤蜊"'],
 7: ['"服務" ',
  '"味道" ',
  '"餐點" ',
  '"義大利麵" ',
  '"調酒" ',
  '"喜歡" ',
  '"推薦" ',
  '"用餐" ',
  '"餐廳" ',
  '"食物"'],
 8: ['"雞湯" ',
  '"服務" ',
  '"夜市" ',
  '"餐廳" ',
  '"歌手" ',
  '"適合" ',
  '"地方" ',
  '"味道" ',
  '

###Visualizer

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook(local=True)
d = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary)
# pyLDAvis.show(d)
pyLDAvis.save_html(d, '/content/drive/Shareddrives/文字探勘/final_project/lda.html')

###Build topic vector for each restaurant

In [ ]:
'''return top topic of each restaurant, stored in list called topics'''
def get_restaurant_topic():
  doc_topics = lda.get_document_topics(corpus) 
  topics = list()
  for i in range(len(doc_topics)):
      print(sorted(doc_topics[i], key=lambda x: x[1], reverse=True))
      topics.append(sorted(doc_topics[i], key=lambda x: x[1], reverse=True)[0][0])
  return topics

In [ ]:
def get_restaurant_words(topics):
  '''given a list of top topic of the restaurants, we'll return corresponding words of the topic'''
  return [topic_words[topic_id] for topic_id in topics]

In [ ]:
topics = get_restaurant_topic()
restaurant_df['topic_id'] = topics
restaurant_words = get_restaurant_words(topics)#so now we have topic words of each restaurant
restaurant_df['topic_words'] = pd.Series(restaurant_words)
restaurant_df

[(4, 0.50287026), (1, 0.22393905), (5, 0.18606527), (22, 0.068400525), (8, 0.018363591)]
[(6, 0.99978966)]
[(3, 0.9998613)]
[(20, 0.99976206)]
[(24, 0.99983114)]
[(7, 0.98883045), (5, 0.010926209)]
[(7, 0.99987036)]
[(5, 0.9674527), (16, 0.016739525), (24, 0.015575606)]
[(8, 0.99972785)]
[(9, 0.9998497)]
[(11, 0.9887324), (17, 0.011129363)]
[(1, 0.44906518), (5, 0.41767633), (22, 0.06710448), (24, 0.038683526), (19, 0.01889806)]
[(12, 0.9992377)]
[(20, 0.5993139), (16, 0.40027183)]
[(4, 0.9997879)]
[(5, 0.9997499)]
[(2, 0.99978364)]
[(3, 0.80357563), (9, 0.19618991)]
[(0, 0.999682)]
[(19, 0.99985546)]
[(7, 0.9997757)]
[(5, 0.9784684), (13, 0.01953104)]
[(18, 0.9997861)]
[(17, 0.9997688)]
[(1, 0.999753)]
[(9, 0.9998015)]
[(3, 0.5281718), (9, 0.4528882), (16, 0.018760236)]
[(11, 0.6821149), (4, 0.31612214)]
[(7, 0.8539256), (9, 0.13538696), (8, 0.010526875)]
[(11, 0.63003206), (4, 0.3454907), (3, 0.024341796)]
[(11, 0.999856)]
[(13, 0.99984926)]
[(5, 0.4883488), (13, 0.2990674), (4, 0.18

,restaurant_id,restaurant_name,length,ave_rating,popularity,topic_id,topic_words
0,0,茱莉金牛排餐酒館,634,4.5,967,4,"[""服務"" , ""牛排"" , ""餐點"" , ""用餐"" , ""餐廳"" , ""環境"" , ""麵包..."
1,1,Okaeri お帰り 你回來啦拉麵,840,4.3,2769,6,"[""拉麵"" , ""湯頭"" , ""柑橘"" , ""叉燒"" , ""味道"" , ""豚骨"" , ""喜歡..."
2,2,和牛涮 日式鍋物放題 台北忠孝東店,840,4.8,21588,3,"[""服務"" , ""新鮮"" , ""用餐"" , ""店員"" , ""態度"" , ""親切"" , ""品質..."
3,3,狗一下居食酒屋 忠孝店,840,4.8,18200,20,"[""服務"" , ""新鮮"" , ""海鮮"" , ""用餐"" , ""海膽"" , ""餐點"" , ""東西..."
4,4,蘇阿姨比薩屋,840,4.1,8271,24,"[""炸雞"" , ""披薩"" , ""牛肉麵"" , ""湯圓"" , ""味道"" , ""牛肉"" , ""紅..."
...,...,...,...,...,...,...,...
101,101,北門鳳李冰,840,4.6,2279,14,"[""芋頭"" , ""花生"" , ""冰品"" , ""鳳梨"" , ""古早味"" , ""味道"" , ""鳳..."
102,102,蛤?! Huh Pot,840,4.8,4614,3,"[""服務"" , ""新鮮"" , ""用餐"" , ""店員"" , ""態度"" , ""親切"" , ""品質..."
103,103,小李子清粥小菜,840,3.8,5029,5,"[""餐點"" , ""服務"" , ""用餐"" , ""環境"" , ""份量"" , ""適合"" , ""味道..."
104,104,温州街蘿蔔絲餅達人,840,4.2,4100,12,"[""蘿蔔絲"" , ""蘿蔔絲餅"" , ""味道"" , ""排隊"" , ""粉圓"" , ""蔥油餅"" ,..."


In [ ]:
#store the complete restaurant_df
restaurant_df.to_csv("/content/drive/Shareddrives/文字探勘/final_project/restaurant_final_df.csv")

###Compute cosine similarity among each of the two restaurants

In [ ]:
from pprint import pprint
from gensim import similarities
index = gensim.similarities.MatrixSimilarity(model[corpus])#lda[corpus] returns topic distribution for each doc; model[corpus] reutrns tfidf vector for each doc
print(model[corpus][0])

[(39, 0.054728959698930435), (45, 0.03853421925344922), (100, 0.06018403233969659), (102, 0.04447051161903972), (183, 0.031394446957406646), (184, 0.028549545177080297), (190, 0.09663965334761516), (217, 0.0490370180245348), (218, 0.054034032706594), (259, 0.04278019799502836), (270, 0.035440647016674), (284, 0.035440647016674), (326, 0.058280793201519736), (334, 0.0394716831814869), (348, 0.046362785021080606), (357, 0.03417976783745757), (387, 0.046362785021080606), (409, 0.14088140159592302), (479, 0.12995066002491953), (504, 0.058029784315336994), (588, 0.03407146418473911), (589, 0.03520656032310642), (598, 0.41725356144185793), (599, 0.06954417753162091), (602, 0.046362785021080606), (623, 0.09310932620070816), (632, 0.03407146418473911), (633, 0.106321941050022), (638, 0.026469625348219155), (669, 0.08364552958872287), (689, 0.04196388986476405), (690, 0.03525481434534738), (698, 0.07940887604465748), (707, 0.6558916771113946), (708, 0.028549545177080297), (709, 0.04025182902740

In [ ]:
'''Store similarity matrix index'''
with open('/content/drive/Shareddrives/文字探勘/final_project/similarity.pkl', 'wb') as f:
  pickle.dump(index, f)

####Load similarity index

In [ ]:
index = pickle.load(open('/content/drive/Shareddrives/文字探勘/final_project/similarity.pkl','rb'))

###Restaurant ranking based on a given topic keyword

In [ ]:
input_keyword = "法式 酥烤 嚼勁 松露"

In [ ]:
def preprocess_input(input_keyword):
  to_token = input_keyword.split()
  bow_vector = dictionary.doc2bow(to_token)
  bow_vector_ordered = sorted(bow_vector, key=lambda x: x[1], reverse=True)# returns (word id, freq of word)
  # print(bow_vector_ordered[:10])
  topic_distrib = lda[bow_vector_ordered]#topic distribution of the input_keyword
  # print(topic_distrib)

  return bow_vector_ordered

In [ ]:
def retrieve(preference):
  '''
  input: latent preference vector
  output: top five similar restaurants
  '''
  sims = index[preference]
  recommended_rest_id = sims.argsort()[-2:-7:-1]

  recommended = [restaurant_df[restaurant_df['restaurant_id'] == id]['restaurant_name'].tolist()[0] for id in recommended_rest_id]#retrieve five restaurants

  #Sorting by ave_rating
  recommended_ordered_index = np.argsort([restaurant_df[restaurant_df['restaurant_name'] == restaurant]['ave_rating'].tolist()[0] for restaurant in recommended])
  if len(recommended_ordered_index) > 5: recommended_ordered_index = recommended_ordered_index[:5]
  recommended = [recommended[i] for i in recommended_ordered_index]
  return recommended

In [ ]:
input_vector = preprocess_input(input_keyword)
print('Top relevant restaurants', '-----', retrieve(input_vector), sep='\n')

Top relevant restaurants
-----
['WUnique 無二法式甜點', 'JAPOLI義大利餐酒館 新生店', '茱莉金牛排餐酒館', 'Mosun 墨賞新鐵板料理餐廳', '豬跳舞小餐館']


###Recommend restaurants based on selected restaurant's corresponding topic

In [ ]:
def recommend(selected_restaurant):
  topic = restaurant_df[restaurant_df['restaurant_name'] == selected_restaurant]['topic_id'].tolist()[0]
  # print(restaurant_df.groupby('topic_id').get_group(topic).sort_values(['ave_rating'], ascending=False)['restaurant_name'])
  restaurant_lst = restaurant_df.groupby('topic_id').get_group(topic).sort_values(['ave_rating'], ascending=False)['restaurant_name'].tolist()
  restaurant_lst = [r for r in restaurant_lst if r != selected_restaurant]
  if len(restaurant_lst) > 5: restaurant_lst = restaurant_lst[:5]
  return restaurant_lst

In [ ]:
selected_restaurant = '茱莉金牛排餐酒館'
recommended_restaurants = recommend(selected_restaurant)
print('Top recommended restaurants', '-----', recommended_restaurants, sep='\n')

Top recommended restaurants
-----
['TASTy 西堤牛排 台北復興南店', '沾美西餐廳', '雅室牛排 仁愛圓環店']
